# This colab is used to try different types of KGE implementation libs following this [link](https://memgraph.com/blog/best-python-packages-tools-for-knowledge-graphs)

# Ampligraph

In [ ]:
! pip install ampligraph
! pip uninstall tensorflow
! pip install "tensorflow>=1.14.0,<2.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached importlib_metadata-4.2.0-py3-none-any.whl (16 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.0.0
    Uninstalling importlib-metadata-5.0.0:
      Successfully uninstalled importlib-metadata-5.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 4.2.0 which is incompatible.
gym 0.25.2 requires importlib-metadata>=4.8.0; python_version < "3.10", but you have importlib-metadata 4.2.0 which is incompatible.


Found existing installation: tensorflow 2.11.0
Uninstalling tensorflow-2.11.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.11.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.11.0
y
y
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 42.8 MB/s 
     |████████████████████████████████| 3.8 MB 48.1 MB/s 
     |████████████████████████████████| 503 kB 64.2 MB/s 
     |████████████████████████████████| 20.1 MB 58.4 MB/s 
     |████████████████████████████████

## Read the data

In [ ]:
import numpy as np
import pandas as pd
import ampligraph

ampligraph.__version__

'1.4.0'

In [ ]:
import requests
from ampligraph.datasets import load_from_csv

url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/GoT.csv'
open('GoT.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'GoT.csv', sep=',')
print(X[:5,:])
X.shape

[['Smithyton' 'SEAT_OF' 'House Shermer of Smithyton']
 ['House Mormont of Bear Island' 'LED_BY' 'Maege Mormont']
 ['Margaery Tyrell' 'SPOUSE' 'Joffrey Baratheon']
 ['Maron Nymeros Martell' 'ALLIED_WITH'
  'House Nymeros Martell of Sunspear']
 ['House Gargalen of Salt Shore' 'IN_REGION' 'Dorne']]


(3175, 3)

Get all unique entity on column 0 and 2
Get all relations on column 1

In [ ]:
entities = np.unique(np.concatenate([X[:,0],X[:,2]]))
print('Entities: ' + str(entities.shape))
relations = np.unique(X[:,1])
print('Relations: ' + str(relations.shape))

Entities: (2050,)
Relations: (10,)


## Preprocessing data

In [ ]:
from ampligraph.evaluation import train_test_split_no_unseen 
X_train, X_test = train_test_split_no_unseen(X, test_size= int(entities.shape[0]* 0.05)) 

In [ ]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (3073, 3)
Test set size:  (102, 3)


## Training model

In [ ]:
from ampligraph.latent_features import ComplEx

In [ ]:
positives_filter = X

In [ ]:
model = ComplEx(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train, early_stopping = False)

Average ComplEx Loss:   0.017605: 100%|██████████| 200/200 [03:24<00:00,  1.02s/epoch]


## Evaluation

In [ ]:
from ampligraph.evaluation import evaluate_performance

In [ ]:
X_unseen = np.array([
    ['House Stark of Winterfell', 'IN_REGION', 'The North'],
    ['House Stark of Winterfell', 'IN_REGION', 'The North'],
])

In [ ]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))
unseen_filter

array([['Walton Frey', 'ALLIED_WITH', 'House Frey of the Crossing'],
       ["House Flint of Widow's Watch", 'SWORN_TO',
        'House Bolton of the Dreadfort'],
       ['House Middlebury', 'IN_REGION', 'The Reach'],
       ...,
       ['House Estermont of Greenstone', 'IN_REGION', 'The Stormlands'],
       ['Jeyne Lydden', 'ALLIED_WITH', 'House Lydden of Deep Den'],
       ['Serala of Myr', 'ALLIED_WITH', 'House Darklyn of Duskendale']],
      dtype='<U44')

In [ ]:
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)

100%|██████████| 2/2 [00:00<00:00, 21.68it/s]


In [ ]:
scores = model.predict(X_unseen)
scores

array([2.5718932, 2.5718932], dtype=float32)

In [ ]:
from scipy.special import expit
probs = expit(scores)
probs

array([0.92903066, 0.92903066], dtype=float32)

In [ ]:
np.squeeze(scores)

array([2.5718932, 2.5718932], dtype=float32)

In [ ]:
np.squeeze(probs)

array([0.92903066, 0.92903066], dtype=float32)

In [ ]:
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")

,statement,rank,score,prob
0,House Stark of Winterfell IN_REGION The North,10,2.571893,0.929031
1,House Stark of Winterfell IN_REGION The North,10,2.571893,0.929031


# VI Spacy

In [ ]:
! pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/vi_core_news_lg/dist/vi_core_news_lg-0.0.1.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 254.5 MB 34 kB/s 
     |████████████████████████████████| 5.8 MB 3.8 MB/s 
     |████████████████████████████████| 10.1 MB 41.9 MB/s 
     |████████████████████████████████| 660 kB 52.5 MB/s 
  Created wheel for vi-core-news-lg: filename=vi_core_news_lg-0.0.1-py3-none-any.whl size=254513618 sha256=b467871a01205a81fb354d877f36d6deb8fbf4ae57f0e2c94219fa072b8cb59f
  Stored in directory: /root/.cache/pip/wheels/e4/d9/90/dcbb25186a2c3335b17bc675f8dddcb43ecdfa4b400e8c91b0
Successfully built vi-core-news-lg
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
  Attemptin

In [ ]:
! pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 3.1 MB/s 
     |████████████████████████████████| 965 kB 43.5 MB/s 


In [191]:
import spacy
nlp = spacy.load('vi_core_news_lg')
questions = nlp('Hành vi gây tai nạn giao thông không dừng lại, không giữ nguyên hiện trường, bỏ trốn không đến trình báo với cơ quan có thẩm quyền, không tham gia cấp cứu người bị nạn bị xử phạt vi phạm hành chính như thế nào?'.lower())
answers = [
    'Hành động dẫn đến nạn giao thông và không dừng lại, không giữ nguyên hiện trạng, trốn đi và không đến trình báo với công an sẽ bị xử phạt dựa trên mức độ nghiêm trọng của người bị nạn',
    'Hành vi không chấp hành hiệu lệnh của đèn tín hiệu giao thông; không chấp hành hiệu lệnh, hướng dẫn của người điều khiển giao thông hoặc người kiểm soát giao thông bị xử phạt vi phạm hành chính như thế nào?',
    'Hành vi điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở và không chấp hành yêu cầu kiểm tra về nồng độ cồn của người thi hành công vụ bị xử phạt vi phạm hành chính như thế nào?'
    'Bấm còi, rú ga liên tục; bấm còi trong thời gian từ 22 giờ đến 5 giờ, bấm còi hơi, sử dụng đèn chiếu xa trong đô thị và khu đông dân cư, trừ các xe được quyền ưu tiên đang đi làm nhiệm vụ theo quy định của Luật này',
    'Phạt tiền từ 800.000 đồng đến 1.000.000 đồng đối với người điều khiển xe bấm còi, rú ga liên tục; bấm còi hơi, sử dụng đèn chiếu xa trong đô thị, khu đông dân cư, trừ các xe ưu tiên đang đi làm nhiệm vụ theo quy định (điểm b khoản 3 Điều 5). Vi phạm quy định trên mà gây tai nạn giao thông thì bị tước quyền sử dụng Giấy phép lái xe từ 02 tháng đến 04 tháng (điểm c khoản 11 Điều 5)',
    'Phạt tiền từ 100.000 đồng đến 200.000 đồng đối với người điều khiển xe bấm còi trong thời gian từ 22 giờ ngày hôm trước đến 05 giờ ngày hôm sau, sử dụng đèn chiếu xa trong đô thị, khu đông dân cư, trừ các xe ưu tiên đang đi làm nhiệm vụ theo quy định (điểm n khoản 1 Điều 6). Vi phạm quy định trên mà gây tai nạn giao thông thì bị tước quyền sử dụng Giấy phép lái xe từ 02 tháng đến 04 tháng (điểm c khoản 10 Điều 6).'
    'Phạt tiền từ 3.000.000 đồng đến 5.000.000 đồng đối với người điều khiển xe dừng xe, đỗ xe trên đường cao tốc không đúng nơi quy định; không có báo hiệu để người lái xe khác biết khi buộc phải dừng xe, đỗ xe trên đường cao tốc không đúng nơi quy định (điểm a khoản 6 Điều 7)',
    'Phạt tiền từ 800.000 đồng đến 1.000.000 đồng đối với người Điều khiển xe thực hiện hành vi Lùi xe ở đường một chiều, đường có biển “Cấm đi ngược chiều”, khu vực cấm dừng, trên phần đường dành cho người đi bộ qua đường, nơi đường bộ giao nhau, nơi đường bộ giao nhau cùng mức với đường sắt, nơi tầm nhìn bị che khuất; lùi xe không quan sát hoặc không có tín hiệu báo trước',
    'Sử dụng đèn chiếu xa khi tránh xe đi ngược chiều (điểm e khoản 3 Điều 7)',
    'Tín hiệu vàng là phải dừng lại trước vạch dừng, trừ trường hợp đã đi quá vạch dừng thì được đi tiếp; trong trường hợp tín hiệu vàng nhấp nháy là được đi nhưng phải giảm tốc độ, chú ý quan sát, nhường đường cho người đi bộ qua đường',
    'Tại Điều 30, 31 Luật Giao thông đường bộ năm 2008 quy định: Với người điều khiển xe mô tô hai bánh, xe mô tô ba bánh, xe gắn máy, xe đạp không được sử dụng ô, điện thoại di động, thiết bị âm thanh, trừ thiết bị trợ thính.',
    'Việt nam vô địch'
    ]
for i in range(len(answers)):
  print(i, questions.similarity(nlp(answers[i].lower())))




0 0.9536993427183409
1 0.8675100818728579
2 0.8511079616565871
3 0.8519718769509561
4 0.8533762837840726
5 0.8478474227061701
6 0.7466200907529629
7 0.8004977001065798
8 0.7967541752331051
9 0.34632884693872945


# Keyphrase tagging

In [190]:
import spacy
import os
import pandas as pd
from pandas import DataFrame
from spacy.language import Language
nlp: Language = spacy.load('vi_core_news_lg')
path = 'keyphrase-extraction.csv'


def read_csv(file_path: str) -> DataFrame:
    data = pd.read_csv(file_path)
    return data


In [192]:
data = read_csv(path)
data

,No,Question,Keyphrase
0,1,Nguyên tắc hoạt động giao thông đường b...,- nguyên tắc\n- đường bộ
1,2,Các hành vi bị nghiêm cấm thực hiện theo Luật ...,- hành vi\n- nghiêm cấm\n- Luật giao thông đườ...
2,3,Quy tắc chung khi tham gia giao thông đường bộ...,- quy tắc\n- đường bộ\n- Luật giao thông đường bộ
3,4,Theo quy định của Luật Giao thông đường bộ thì...,- Luật giao thông đường bộ\n- hệ thống báo hiệ...
4,5,Tốc độ và khoảng cách giữa các xe cơ giới khi ...,- tốc độ\n- khoảng cách \n- xe cơ giới\n- mức ...
...,...,...,...
95,96,Hành vi điều khiển xe trên đường mà trong máu ...,- hành vi\n- điều khiển\n- xe \n- nồng độ cồn\...
96,97,Việc vượt đèn đỏ với từng loại phương tiện sẽ ...,- vượt\n- đèn đỏ\n- phương tiện\n- phạt
97,98,Người điểu khiển phương tiện khi lùi xe cần ph...,- người điều khiển\n- phương tiện\n- lùi xe\n-...
98,99,Luật giao thông đường bộ năm 2008 quy định như...,- quy định\n- tín hiệu\n- đèn giao thông\n- xử...


In [193]:
def split_keyphrase(row):
  row = str(row).replace('-','')
  
  res= list(filter(lambda keyphrase: len(keyphrase)!=0 ,list(map(lambda item: item.strip().lower(),row.split('\n')))))

  return res
raw_keyphrases = data['Keyphrase']
raw_keyphrases = raw_keyphrases.map(lambda row: split_keyphrase(row))
data['Cleanse Keyphrases'] = raw_keyphrases
data

,No,Question,Keyphrase,Cleanse Keyphrases
0,1,Nguyên tắc hoạt động giao thông đường b...,- nguyên tắc\n- đường bộ,"[nguyên tắc, đường bộ]"
1,2,Các hành vi bị nghiêm cấm thực hiện theo Luật ...,- hành vi\n- nghiêm cấm\n- Luật giao thông đườ...,"[hành vi, nghiêm cấm, luật giao thông đường bộ..."
2,3,Quy tắc chung khi tham gia giao thông đường bộ...,- quy tắc\n- đường bộ\n- Luật giao thông đường bộ,"[quy tắc, đường bộ, luật giao thông đường bộ]"
3,4,Theo quy định của Luật Giao thông đường bộ thì...,- Luật giao thông đường bộ\n- hệ thống báo hiệ...,"[luật giao thông đường bộ, hệ thống báo hiệu, ..."
4,5,Tốc độ và khoảng cách giữa các xe cơ giới khi ...,- tốc độ\n- khoảng cách \n- xe cơ giới\n- mức ...,"[tốc độ, khoảng cách, xe cơ giới, mức phạt]"
...,...,...,...,...
95,96,Hành vi điều khiển xe trên đường mà trong máu ...,- hành vi\n- điều khiển\n- xe \n- nồng độ cồn\...,"[hành vi, điều khiển, xe, nồng độ cồn, vượt qu..."
96,97,Việc vượt đèn đỏ với từng loại phương tiện sẽ ...,- vượt\n- đèn đỏ\n- phương tiện\n- phạt,"[vượt, đèn đỏ, phương tiện, phạt]"
97,98,Người điểu khiển phương tiện khi lùi xe cần ph...,- người điều khiển\n- phương tiện\n- lùi xe\n-...,"[người điều khiển, phương tiện, lùi xe, tuân t..."
98,99,Luật giao thông đường bộ năm 2008 quy định như...,- quy định\n- tín hiệu\n- đèn giao thông\n- xử...,"[quy định, tín hiệu, đèn giao thông, xử phạt, ..."


## Tagging keyphrases in a question
> A question could have multiple keyphrases, each keyphrase is a combination of token. We have to tag all these token

> Keyphrase tagging is stored as 2D scala like below

```
[
  ['nguyên_tắc|N'], 
  ['vượt|V', 'xe|N'], 
  ['trường_hợp|N'], 
  ['không|R', 'được|V', 'vượt|V', 'xe|N'], 
  ['vượt|V', 'xe|N'], 
  ['không|R', 'đúng|A', 'quy_định|N'], 
  ['xử_phạt|V']
]

```
> We can see here there are 7 keyphrase, each keyphrase is the combination of token are tagged

In [208]:
from typing import List
def tagging_keyphrase(keyphrases: List[str]):
  result = []
  for keyphrase in keyphrases:
    tag_result = []
    doc = nlp(keyphrase)
    for token in doc:
        tag_result.append(token.text+'|'+token.tag_)
    result.append(tag_result)
  print(result)
  return result

In [210]:
keyphrases = [keyphrases for keyphrases in data['Cleanse Keyphrases']]
data['Keyphrases Tagging'] = [tagging_keyphrase(keyphrase) for keyphrase in keyphrases]

[['nguyên_tắc|N'], ['đường_bộ|N']]
[['hành_vi|N'], ['nghiêm_cấm|V'], ['luật|N', 'giao_thông|N', 'đường_bộ|N', 'năm|N', '2008|M']]
[['quy_tắc|N'], ['đường_bộ|N'], ['luật|N', 'giao_thông|N', 'đường_bộ|N']]
[['luật|N', 'giao_thông|N', 'đường_bộ|N'], ['hệ_thống|N', 'báo_hiệu|V'], ['gồm|V'], ['chấp_hành|V'], ['báo_hiệu|V', 'đường_bộ|N']]
[['tốc_độ|N'], ['khoảng_cách|N'], ['xe|N', 'cơ_giới|N'], ['mức|N', 'phạt|V']]
[['nguyên_tắc|N'], ['vượt|V', 'xe|N'], ['trường_hợp|N'], ['không|R', 'được|V', 'vượt|V', 'xe|N'], ['vượt|V', 'xe|N'], ['không|R', 'đúng|A', 'quy_định|N'], ['xử_phạt|V']]
[['chuyển_hướng|V'], ['không|R', 'đúng|A', 'quy_định|N'], ['xử_phạt|V']]
[['tai_nạn|N', 'giao_thông|N'], ['người|N', 'điều_khiển|V', 'phương_tiện|N'], ['lùi|V', 'xe|N'], ['người|N', 'điều_khiển|V', 'phương_tiện|N'], ['lùi|V', 'xe|N'], ['quy_định|N']]
[['tránh|V', 'xe|N'], ['ngược|A', 'chiều|N'], ['vi_phạm|V'], ['quy_định|N'], ['xử_phạt|V']]
[['dừng|V', 'đỗ|V', 'xe|N'], ['quy_định|N'], ['vi_phạm|V'], ['xử_phạt|V']]